(sec:setup-git)=
# Git環境の設定

Gitとは、プログラミングのソースコード等の更新履歴を管理するためのバージョン管理システム (VCS, Version Cotrol System)の一種です。バージョン管理システムにはGit以外にもSubversionやMercurialなどがある。

Gitを利用するには、
1. [GitHub](https://github.com/)や[BitBucket](https://bitbucket.org)等のウェブサービスを使う
2. 自分でGitサーバーを立てて使う

の2通りの方法が考えらるが、よほど機密性の高いソースコードを扱う等の場合でなければ、ウェブサービスを使う方法で問題ない。

本節では、GitHubを用いてGitレポジトリの作成とソースコードの管理を行う方法を紹介する。

## Gitについて

Gitは**分散型バージョン管理システム**であり、一つのプロジェクトに関連するコードを複数のユーザが別々に編集することを想定して設計されている。

Gitにおいては、ここのユーザが作成した変更が**コミット**という形で管理され、異なるユーザが作成したコミットを**マージ**することにより、コードを更新していく。

一方、Git以前に主流であったSubversionは**中央集権的なバージョン管理システム**であり、とあるユーザがソースコードを編集している間は、サーバ上のソースコードの更新がロックされるという仕組みであった。そのため、複数の異なるユーザが同時にソースコードを編集するようなプロジェクトには不向きであった。

現在、多くのソフトウェアプロジェクトは、システムの大規模化に伴い、複数のユーザで共同開発されるのが一般的になっており、Gitは現在のプログラミング・パラダイムにより即した仕組みであると言える。

```{admonition} Gitの特長
:class: note

- 分散型であるために複数のユーザが同時にコードを編集可能
- 各ユーザの編集内容はローカル環境で「コミット」という単位で管理される
- 複数のユーザが行った編集が「マージ」されることでコードが更新されていく
```

## GitHubアカウントの作成

公式アナウンスに従ってアカウントを作成しよう。ユーザ名は変更できないので慎重に考えること。

<https://docs.github.com/ja/get-started/quickstart/creating-an-account-on-github>

## Gitリポジトリの作成

GitHubを用いてバージョン管理を開始するには**リポジトリ**を作成する必要がある。

本来、リポジトリを作成するには、Gitサーバ上で

```shell
git init --bare
```

というコマンドを実行する必要があるのだが、GitHubを用いる場合には、ウェブ上の操作によりGitリポジトリを作成することができる。

まずは、各自のGitHubのトップページ (`https://github.com/[ユーザ名]`)にアクセスし、右上の「＋」ボタンから「Create New Repository」を選ぶ。

## ローカル環境の準備

### Windows

Windowsは、初期状態ではGit環境が用意されていないので、別途ソフトウェアをインストールする必要がある。

最も一般的なものは[Git for Windows](https://gitforwindows.org/)なのだが、こちらを使う場合には、Gitコマンドに環境変数のパスを通すなど、余計な設定が必要になるため、インストールの際に注意が必要。

環境変数のパスを通す設定をし損ねた場合には、自分で環境変数のパスを通すか、(慣れないと使い勝手が悪い)Git Bashというシステムを使うことになる。

そこでおすすめしたいのが、PowerShell上で動く、ソフトウェア管理ツールである[Chocolatey](https://chocolatey.org/)である。Chocolateyをインストールするための(おすすめ)手順は以下の通り。

1. オープンソース版の[PowerShell](https://github.com/PowerShell/PowerShell/releases/tag/v7.4.1)をインストールする。

2. PowerShellを**管理者権限で**開き、[Install Chocolatey](https://chocolatey.org/install#individual)のページ中段にある以下のコマンドを実行する。

```powershell
Set-ExecutionPolicy Bypass -Scope Process -Force; [System.Net.ServicePointManager]::SecurityProtocol = [System.Net.ServicePointManager]::SecurityProtocol -bor 3072; iex ((New-Object System.Net.WebClient).DownloadString('https://community.chocolatey.org/install.ps1'))
```

3. PowerShellを開き、`choco list -l`とタイプしてEnterを押す。正しくコマンドが認識されていれば、現在のChocolateyのバージョンが表示される。

ここまででChocolateyのインストールは完了なので、最後にChocolateyを使ってGitをインストールする ([参考ページ](https://community.chocolatey.org/packages/git))。

```powershell
choco install git
```

インストール完了後、PowerShell上で`git --version`とタイプしてEnterを押す。正しくコマンドが認識されていれば、現在のGitのバージョンが表示される。

### Mac / Linux

特に準備をする必要はなく、ターミナルから`git`コマンドが使用できる。念のため、ターミナルを開き `git --version`とタイプしてEnterを押す。正しくコマンドが認識されていれば、現在のGitのバージョンが表示される。

### GUIを通してGitを使う (非推奨)

もしGUIを用いて、視覚的にバージョン管理を行いたい場合は、[GitHub Desktop](https://desktop.github.com/)や[Sourcetree](https://www.sourcetreeapp.com/)などを使うと良い。

しかし、GUIを使っていると、なかなか融通が利かない部分もあり、またGitのコマンドをなかなか覚えないので、基本はターミナルからコマンドを実行することを強く推奨する。

```{admonition} Gitのインストール
:class: note

- WindowsはGit for Windowsをインストールするか、Chocolateyを使ってGitをインストール
- MacやLinuxの場合には特段、インストールをしなくてもGitがインストールされている
- Gitをローカル環境で操作する場合にはシェル環境 (PowerShellやターミナル等)を使うことを推奨
```

## Gitの基本コマンド

Gitでまず覚えるべきコマンドは**クローン** (`git clone`)、**コミット** (`git commit`)、そして**プッシュ** (`git push`)の3つである。

### クローン

### コミット

### プッシュ

## ソースコードの編集・管理